In [ ]:
 !nvidia-smi

Fri Sep 15 01:00:05 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P8    11W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/TCC/tck/data/alvaro_maia_marceio/2023-09-03

/content/drive/.shortcut-targets-by-id/1kA-euLAXeyvRnYVxUC9fmOnFh8p-vEpK/TCC/tck/data/alvaro_maia_marceio/2023-09-03


In [ ]:
!ls

'2023-09-03 08-05-49.mkv'  '2023-09-03 20-16-24.mkv'   yolov8x.pt
'2023-09-03 14-30-02.mkv'   video_saida.mp4


In [ ]:
import os
HOME = os.getcwd()
print(HOME)

/content/drive/.shortcut-targets-by-id/1kA-euLAXeyvRnYVxUC9fmOnFh8p-vEpK/TCC/tck/data/alvaro_maia_marceio/2023-09-03


In [ ]:
!ls {HOME}

'2023-09-03 08-05-49.mkv'  '2023-09-03 20-16-24.mkv'   yolov8x.pt
'2023-09-03 14-30-02.mkv'   video_saida.mp4


In [ ]:
SOURCE_VIDEO_PATH = f"{HOME}/2023-09-03 08-05-49.mkv"



In [ ]:
# Pip install method (recommended)

%pip install ultralytics

from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.0.179 🚀 Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 26.3/78.2 GB disk)


## Install Roboflow Supervision

In [ ]:
#!yolo task=detect mode=track model=yolov8x.pt conf=0.25 source='/content/drive/MyDrive/TCC/tck2/mario2.mp4'

## Tracking utils

Unfortunately, we have to manually match the bounding boxes coming from our model with those created by the tracker.

In [ ]:
!pip install supervision==0.1.0

In [ ]:
from IPython import display
display.clear_output()


import supervision
print("supervision.__version__:", supervision.__version__)

from collections import defaultdict
import cv2
import numpy as np

from supervision.geometry.dataclasses import Point
from supervision.video.dataclasses import VideoInfo
from supervision.video.source import get_video_frames_generator
from supervision.notebook.utils import show_frame_in_notebook


supervision.__version__: 0.1.0


## Load pre-trained YOLOv8 model

In [ ]:
# settings
MODEL = "yolov8x.pt"

In [ ]:
from ultralytics import YOLO

model = YOLO(MODEL)
model.fuse()

YOLOv8x summary (fused): 268 layers, 68200608 parameters, 0 gradients


## Predict and annotate single frame

In [ ]:
# dict maping class_id to class_name
CLASS_NAMES_DICT = model.model.names
# class_ids of interest - car, motorcycle, bus and truck

CLASS_ID = [2, 3, 5, 7]
CLASS_ID_NAMES =['car', 'motorcycle',' bus' , 'truck']

#SOURCE_VIDEO = f"{SOURCE_VIDEO_PATH}{arquivos[0]}"


In [ ]:
import math
import numpy as np

def distancia2d(x1, y1, x2, y2):
  a = x2 - x1
  b = y2 - y1
  c = math.sqrt(math.pow(a, 2) + math.pow(b, 2))
  return c


def distancia_entre_pontos(ponto1, ponto2):
    return np.linalg.norm(np.array(ponto1) - np.array(ponto2))


In [ ]:
import cv2
import numpy as np

import matplotlib.pyplot as plt


def line_to_points(line):
    x1, y1, x2, y2 = line
    points = []

    # Usando a equação da reta para encontrar os pontos intermediários
    dx = abs(x2 - x1)
    dy = abs(y2 - y1)

    if dx >= dy:
        steps = dx
    else:
        steps = dy

    if steps == 0:
        return [(x1, y1)]

    x_increment = (x2 - x1) / steps
    y_increment = (y2 - y1) / steps

    x, y = x1, y1
    for _ in range(steps + 1):
        points.append((int(x), int(y)))
        x += x_increment
        y += y_increment

    return points



def pontos_mais_proximo(line, x1,y1):
    menor_distancia = float('inf')
    ponto_mais_proximo = None

    points = line_to_points(line)

    for ponto in points:
              x, y = ponto
              d = distancia2d(x, y, x1, y1)
              if d < menor_distancia:
                  menor_distancia = d

                  ponto_mais_proximo = ponto
    return ponto_mais_proximo, menor_distancia


In [ ]:
import cv2

line = (250, 1050, 1920, 730)

lineEsquerda = (100, 320, 200, 1050)
lineDireita =  (830, 320, 1920, 730)

lineDireitaCima = (800, 320,1000, 450) (275, 320), (350,500)
lineEsquerdaCima = (275, 320,350,500)

generator = get_video_frames_generator(SOURCE_VIDEO_PATH)
for i in range(178):
    iterator = iter(generator)
    frame = next(iterator)


results = model.track(frame, persist=True)

boxes= results[0].boxes.xyxy.cpu().numpy()
track_ids= results[0].boxes.id.int().cpu().tolist()
class_ids= results[0].boxes.cls.cpu().numpy().astype(int)



menor_distancia = float('inf')
ponto_mais_proximo = None



for box, track_id, class_id in zip(boxes, track_ids, class_ids):

          x1, y1, x2, y2 = box



          if y1 < Y_MED :
             if x1 > X_MED:
                closest_point, min_distance = pontos_mais_proximo(lineDireitaCima, x1, y2)

                print(track_id,x1,y1,min_distance)
                print(closest_point)
             else:
                closest_point, min_distance = pontos_mais_proximo(lineDireitaCima, x1, y2)

                print(track_id,x1,y1,min_distance)
                print(closest_point)



frame = results[0].plot()

#Linha de baixo


cv2.line(frame, (200, 1050), (1920, 730), (255,0,0), 2)

#Linha de direita Cima
cv2.line(frame, (800, 320), (1000, 450), (0,255,0), 2)
#Linha de esquerda Cima  (275, 320)
cv2.line(frame, (275, 320), (350,500), (0,255,0), 2)
#Esquerda Baixo
cv2.line(frame, (100, 320), (200, 1050), (0,255,0), 2)
#Linha de cima
#verde
cv2.line(frame, (100, 320), (800, 320), (0,255,0), 2)
#amarela
cv2.line(frame, (100, 320), (450, 320), (0,255,255), 3)
#azul
cv2.line(frame, (100, 320), (275, 320), (255,255,0), 4)
#branca
cv2.line(frame, (450, 320), (625, 320), (255,255,255), 5)
#cv2.imwrite('/content/drive/frame.jpg', frame)

%matplotlib inline
show_frame_in_notebook(frame, (16, 16))

<>:8: SyntaxWarning: 'tuple' object is not callable; perhaps you missed a comma?
<>:8: SyntaxWarning: 'tuple' object is not callable; perhaps you missed a comma?
<ipython-input-21-2b66a5eba927>:8: SyntaxWarning: 'tuple' object is not callable; perhaps you missed a comma?
  lineDireitaCima = (800, 320,1000, 450) (275, 320), (350,500)


TypeError: ignored

In [ ]:

line2 = (450, 320,800, 320)
points = line_to_points(line2)


In [ ]:


# Encontre os pontos extremos (mínimo e máximo) ao longo do eixo x
min_x = min(points, key=lambda ponto: ponto[0])
max_x = max(points, key=lambda ponto: ponto[0])

# Calcule o ponto central da reta
ponto_central = ((min_x[0] + max_x[0]) / 2, min_x[1])

print("O ponto central da reta é:", ponto_central)

O ponto central da reta é: (625.0, 320)


In [ ]:
L1_start = Point(0, 320)
L1_end = Point(800, 320)

L2_start = Point(200, 1050)
L2_end = Point(1920, 730)

Y_MED = 450
X_MED = 750

In [ ]:
video_info = VideoInfo.from_video_path(SOURCE_VIDEO_PATH)

In [ ]:
video_info


VideoInfo(width=1920, height=1080, fps=23, total_frames=1466)

In [ ]:
# Obter informações do vídeo de entrada
import cv2

cap = cv2.VideoCapture(SOURCE_VIDEO_PATH)

fps = int(cap.get(cv2.CAP_PROP_FPS))
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Nome do arquivo de vídeo de saída
output_video_path = 'video_saida.mp4'

# Configurar o codec de vídeo e o objeto VideoWriter para gravar o vídeo de saída
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Você pode alterar o codec conforme necessário
out = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))


In [ ]:
def is_close(distance, threshold=20):
    return 0 < distance <= threshold

In [ ]:
from tqdm.notebook import tqdm
import cv2

count = 0
dict_frames ={}
dict_framesRIGTH ={}
count2 =0
count3 =0
dict_count  = {}
t2 = []

generator = get_video_frames_generator(SOURCE_VIDEO_PATH)
video_info = VideoInfo.from_video_path(SOURCE_VIDEO_PATH)

lineCima =(0, 320,800, 320)
lineBaixo=(200, 1050,1920, 730)
lineDireitaCima = (800, 320,1000, 450)
#lineEsquerda = (100, 320, 200, 1050)
#lineDireita =  (830, 320, 1920, 730)


cap = cv2.VideoCapture(SOURCE_VIDEO_PATH)

# acquire first video frame
while cap.isOpened():
    # Read a frame from the video
    success, frame = cap.read()

    if success:

      results = model.track(frame, persist=True)

      boxes = results[0].boxes.xyxy.cpu().numpy()
      track_ids = results[0].boxes.id.int().cpu().tolist()
      class_ids=results[0].boxes.cls.cpu().numpy().astype(int)



      for box, track_id, class_id in zip(boxes, track_ids, class_ids):
          x1, y1, x2, y2 = box

          if class_id == 0:
             continue

          if track_id not in dict_count:
             dict_count[track_id] = class_id

             if y1 < Y_MED:
                if x1 > X_MED:
                      count2 +=1
                else:
                      count3 +=1


          #Quadrante Inferior
          """
          if y1 > Y_MED:
              closest_point, min_distance = pontos_mais_proximo(lineBaixo, x1, y1)

              if is_close(min_distance):
                  if track_id not in dict_frames:
                      dict_frames[track_id] = [CLASS_ID_NAMES[CLASS_ID.index(class_id)], count]

          else:
              closest_point, min_distance = pontos_mais_proximo(lineCima, x1, y1)
              if is_close(min_distance):
                  if track_id in dict_frames:
                      if len(dict_frames[track_id]) < 3:
                          dict_frames[track_id].append(count)
                      else:
                          dict_frames[track_id][2] = count"""


          #QUADRANTE SUPERIOR DIREITA

          """ closest_point_rigth, min_distance_rigth = pontos_mais_proximo(lineDireita, x1, y1)

                  if is_close(min_distance_rigth):
                      if track_id not in dict_framesRIGTH:
                          dict_framesRIGTH[track_id] = [CLASS_ID_NAMES[CLASS_ID.index(class_id)], count]

             else:
                  closest_point_rigth, min_distance_rigth = pontos_mais_proximo(lineEsquerda, x1, y1)
                  if is_close(min_distance_rigth):
                      if track_id in dict_frames:
                          if len(dict_frames[track_id]) < 3:
                              dict_frames[track_id].append(count)
                          else:
                              dict_frames[track_id][2] = count"""


      #frame = results[0].plot()
      #out.write(frame)

      count+=1


    else:
        break



# Fechar os objetos VideoCapture e VideoWriter
"""
cap.release()
out.release()
"""


0: 384x640 3 cars, 1 traffic light, 63.0ms
Speed: 2.9ms preprocess, 63.0ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 6 cars, 1 traffic light, 61.9ms
Speed: 3.6ms preprocess, 61.9ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 6 cars, 1 traffic light, 58.0ms
Speed: 2.9ms preprocess, 58.0ms inference, 2.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 6 cars, 1 traffic light, 50.8ms
Speed: 3.4ms preprocess, 50.8ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 6 cars, 1 traffic light, 47.8ms
Speed: 3.1ms preprocess, 47.8ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 6 cars, 1 traffic light, 40.0ms
Speed: 3.2ms preprocess, 40.0ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 6 cars, 1 traffic light, 39.6ms
Speed: 3.1ms preprocess, 39.6ms inference, 2.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 6

'\ncap.release()\nout.release()\n'

In [ ]:

count2

86

In [ ]:
count3

58

In [ ]:
!cp video_saida.mp4 /content/drive/MyDrive/TCC/

In [ ]:
OUTPUT = '/content/drive/MyDrive/TCC/tck2/OUTPUTS/alvaro_maia_com_marceio/2023-08-18/09-20-05/'

In [ ]:
import pandas as pd



df6 = pd.DataFrame.from_dict(dict_count, orient='index', columns=['type'])
df6.reset_index(inplace=True)
df6.rename(columns={'index': 'ID'}, inplace=True)

# Especifique o caminho para o arquivo CSV
caminho_arquivo_csv = f"{OUTPUT}data_count.csv"


# Gravar o DataFrame no arquivo CSV
df6.to_csv(caminho_arquivo_csv, index=False)
df1 = pd.DataFrame.from_dict(dict_count, orient='index', columns=['count'])

## Predict and annotate whole video

In [ ]:
import pandas as pd


df = pd.DataFrame.from_dict(dict_frames, orient='index', columns=['Type', 'Frame In', 'Frame Out'])
df.reset_index(inplace=True)
df.rename(columns={'index': 'ID'}, inplace=True)

# Criando o DataFrame a partir da lista de dicionários


# Especifique o caminho para o arquivo CSV
#caminho_arquivo_csv2 = f"{OUTPUT}data_frames.csv"

# Gravar o DataFrame no arquivo CSV
#df.to_csv(caminho_arquivo_csv2, index=False)
#df = pd.DataFrame.from_dict(dict_frames, orient='index', columns=['count'])


In [ ]:
df

In [ ]:
from moviepy.editor import VideoFileClip

def get_video_duration(video_path):
    try:
        video_clip = VideoFileClip(video_path)
        duration_minutes = video_clip.duration / 60.0
        return duration_minutes
    except Exception as e:
        print(f"Erro ao abrir o vídeo: {e}")
        return None

# Substitua 'seu_video.mp4' pelo caminho para o seu vídeo

duration_minutes = get_video_duration(SOURCE_VIDEO_PATH)

if duration_minutes is not None:
    print(f"A duração do vídeo é de {duration_minutes} minutos")


A duração do vídeo é de 21.825833333333332 minutos


In [ ]:
#09-20-05
#14-50-58
#19-58-55
import pandas as pd


duration_minutes = get_video_duration(SOURCE_VIDEO_PATH)

dados = {
    'RUA': ['alvaro_maia_com_marceio'],
    'DATA': ['2023-08-18'],
    'HORA': ['09-20-05'],
    'TEMPO_MIN': [duration_minutes],
    'FPS': [video_info.fps],
    'TAMANHO_TRAJETO': ['18-34'],
    'TOTAL FRAMES': [video_info.total_frames]
}
df = pd.DataFrame(dados)
df.to_csv(f"{OUTPUT}dados_video.csv", index=False)